In [1]:
import cv2, csv
import numpy as np
from keras.models import Sequential
from keras.models import load_model
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils  import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

SIZE = 3500
MODEL_FOLDER = "model/"
WIDTH = 140
HEIGHT = 48
PROCESSED_FOLDER = "processed/"
LABEL_CSV_FILE = 'label.csv'
dic19 = {'2':0, '3':1, '4':2, '5':3, '7':4, '9':5, 'A':6, 'C':7, 'F':8, 'H':9, 'K':10, 'M':11, 'N':12, 'P':13, 'Q':14, 'R':15, 'T':16, 'Y':17, 'Z':18}


Using TensorFlow backend.


In [2]:
def to_onelist(text):
    label_list = []
    for c in text:
        onehot = [0 for _ in range(len(dic19))]
        onehot[dic19[c]] = 1
        label_list.append(onehot)
    return label_list

In [3]:
#creat CNN model
print('Creating CNN model...')
tensor_in = Input((HEIGHT, WIDTH, 3))
tensor_out = tensor_in
tensor_out = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(tensor_out)
tensor_out = MaxPooling2D(pool_size=(2, 2))(tensor_out)
tensor_out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(tensor_out)
tensor_out = MaxPooling2D(pool_size=(2, 2))(tensor_out)
tensor_out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(tensor_out)
tensor_out = BatchNormalization(axis=1)(tensor_out)
tensor_out = MaxPooling2D(pool_size=(2, 2))(tensor_out)
tensor_out = Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = MaxPooling2D(pool_size=(2, 2))(tensor_out)
tensor_out = Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = BatchNormalization(axis=1)(tensor_out)
tensor_out = MaxPooling2D(pool_size=(2, 2))(tensor_out)

tensor_out = Flatten()(tensor_out)
tensor_out = Dropout(0.5)(tensor_out)

tensor_out = [Dense(19, name='digit1', activation='softmax')(tensor_out),\
              Dense(19, name='digit2', activation='softmax')(tensor_out),\
              Dense(19, name='digit3', activation='softmax')(tensor_out),\
              Dense(19, name='digit4', activation='softmax')(tensor_out)]

W0916 10:01:49.891368 4711863744 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0916 10:01:49.911611 4711863744 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0916 10:01:49.919960 4711863744 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0916 10:01:49.991585 4711863744 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Creating CNN model...


W0916 10:01:50.092438 4711863744 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0916 10:01:50.093451 4711863744 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0916 10:01:50.543286 4711863744 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
model = Model(inputs=tensor_in, outputs=tensor_out)
model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])
model.summary()

W0916 10:01:50.637598 4711863744 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 140, 3)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 48, 140, 32)  896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 46, 138, 32)  9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 23, 69, 32)   0           conv2d_2[0][0]                   
____________________________________________________________________________________________

In [5]:
print("Reading training data...")

train_data = np.stack([np.array(cv2.imread(PROCESSED_FOLDER + str(index) + ".jpg"))/255.0 for index in range(1, SIZE + 1)])
traincsv = open(LABEL_CSV_FILE, 'r', encoding = 'utf8')


read_label =  [to_onelist(row[0]) for row in csv.reader(traincsv)]
train_label = [[] for _ in range(4)]
for arr in read_label:
    for index in range(4):
        train_label[index].append(arr[index])
train_label = [arr for arr in np.asarray(train_label)]

print("Reading completed")

Reading training data...
Reading completed


In [6]:
filepath = MODEL_FOLDER + "{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_digit4_acc', verbose=1, save_best_only=False, mode='max')
earlystop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, mode='auto')
tensorBoard = TensorBoard(log_dir = 'logs', histogram_freq = 1)
callbacks_list = [tensorBoard, earlystop, checkpoint]

In [7]:
model = load_model(MODEL_FOLDER + "25-1.10-3.01.hdf5")

W0916 10:01:58.558650 4711863744 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
history = model.fit(train_data, train_label, validation_split=0.2, batch_size=50, epochs=10, verbose=1, callbacks=callbacks_list)

W0916 10:02:01.011471 4711863744 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/callbacks.py:1120: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.

W0916 10:02:01.154010 4711863744 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/callbacks.py:1122: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0916 10:02:01.158941 4711863744 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/callbacks.py:1125: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Train on 2800 samples, validate on 700 samples
Epoch 1/10
2800/2800 [==============================] - 84s 30ms/step - loss: 0.9477 - digit1_loss: 0.1927 - digit2_loss: 0.2598 - digit3_loss: 0.2288 - digit4_loss: 0.2664 - digit1_acc: 0.9396 - digit2_acc: 0.9196 - digit3_acc: 0.9357 - digit4_acc: 0.9261 - val_loss: 1.0153 - val_digit1_loss: 0.2056 - val_digit2_loss: 0.2861 - val_digit3_loss: 0.2782 - val_digit4_loss: 0.2454 - val_digit1_acc: 0.9443 - val_digit2_acc: 0.9214 - val_digit3_acc: 0.9171 - val_digit4_acc: 0.9286

Epoch 00001: saving model to model/01-0.95-1.02.hdf5
Epoch 2/10
2800/2800 [==============================] - 83s 30ms/step - loss: 0.7699 - digit1_loss: 0.1469 - digit2_loss: 0.2247 - digit3_loss: 0.1835 - digit4_loss: 0.2150 - digit1_acc: 0.9518 - digit2_acc: 0.9336 - digit3_acc: 0.9479 - digit4_acc: 0.9379 - val_loss: 1.0061 - val_digit1_loss: 0.2462 - val_digit2_loss: 0.3055 - val_digit3_loss: 0.2483 - val_digit4_loss: 0.2061 - val_digit1_acc: 0.9371 - val_digit2_a

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_history, train, validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
show_train_history(history, 'digit1_acc', 'val_digit1_acc')